<a href="https://colab.research.google.com/github/ayushjain1144/semantic-segmentation-IGCAR/blob/master/bert_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!pip install sentencepiece

In [90]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/igcar_ps/ /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/igcar_ps': File exists


In [91]:
!git clone https://github.com/google-research/bert

fatal: destination path 'bert' already exists and is not an empty directory.


In [92]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")



from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder
% cd ..
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s : \
    %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
    log.info("Using TPU runtime")
    USE_TPU = True
    TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    
    with tf.Session(TPU_ADDRESS) as session:
        log.info('TPU address is ' + TPU_ADDRESS)
        
       
else:
    log.warning('Not connected to TPU runtime')
    USE_TPU = False

2019-06-26 09:21:33,093 :     Using TPU runtime
I0626 09:21:33.093087 140179299624832 interactiveshell.py:2882] Using TPU runtime
2019-06-26 09:21:33,100 :     TPU address is grpc://10.19.255.18:8470
I0626 09:21:33.100322 140179299624832 interactiveshell.py:2882] TPU address is grpc://10.19.255.18:8470


/


In [93]:
%cd /content/
!ls

/content
1.txt  bert  bert_model  gdrive  pretraining_data  sample_data	shards


In [0]:
# vocabulary formation

#MODEL_PREFIX = "tokenizer"
#VOC_SIZE = 32000
#NUM_PLACEHOLDERS = 256
#SUBSAMPLE_SIZE = 12800000

#SPM_COMMAND = ('--input={} --model_prefix={} '
#              '--vocab_size={} --input_sentence_size={} '
#              '--shuffle_input_sentence=true '
#              '--bos_id=-1 --eos_id=-1').format(
#              'out6_pendrive.txt', MODEL_PREFIX,
#               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)
#spm.SentencePieceTrainer.Train(SPM_COMMAND)

In [0]:

from bert import modeling, optimization, tokenization


In [0]:
import bert.run_pretraining
from bert.run_pretraining import input_fn_builder, model_fn_builder


In [0]:
MAX_SEQ_LENGTH = 128
MASKED_LM_PROB = 0.15
MAX_PREDICTIONS = 20
DO_LOWER_CASE = True
PROCESSES = 2
PRETRAINING_DIR = "pretraining_data"

In [98]:
!wc -w 1.txt

682357 1.txt


In [99]:
!mkdir ./shards
!split -a 4 -l 256000 -d '1.txt' ./shards/shard_

mkdir: cannot create directory ‘./shards’: File exists


In [100]:
!ls ./shards/

shard_0000


In [101]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = '/mydrive/bert_uncased/' + BERT_MODEL
print('****** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR)) 
#!ls BERT_PRETRAINED_DIR

****** BERT pretrained directory: /mydrive/bert_uncased/uncased_L-12_H-768_A-12 *****


In [0]:
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'

In [103]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
            "python3 bert/create_pretraining_data.py "
            "--input_file=./shards/{} "
            "--output_file={}/{}.tfrecord "
            "--vocab_file={} "
            "--do_lower_case={} "
            "--max_seq_length={} "
            "--masked_lm_prob={} "
            "--random_seed=108 "
            "--dupe_factors=5 ")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}',
                            PRETRAINING_DIR, '{}',
                            VOCAB_FILE,
                            DO_LOWER_CASE,
                            MAX_PREDICTIONS, MAX_SEQ_LENGTH,
                            MASKED_LM_PROB)

print(XARGS_CMD)

ls ./shards/ | xargs -n 1 -P 2 -I{} python3 bert/create_pretraining_data.py --input_file=./shards/{} --output_file=pretraining_data/{}.tfrecord --vocab_file=/mydrive/bert_uncased/uncased_L-12_H-768_A-12/vocab.txt --do_lower_case=True --max_seq_length=20 --masked_lm_prob=128 --random_seed=108 --dupe_factors=5 


In [0]:
tf.gfile.MkDir(PRETRAINING_DIR)


In [105]:
!$XARGS_CMD

W0626 09:21:55.100078 140447344932736 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:469: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0626 09:21:55.100918 140447344932736 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0626 09:21:55.101079 140447344932736 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0626 09:21:55.101243 140447344932736 deprecation_wrapper.py:119] From /content/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0626 09:21:55.224929 140447344932736 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0626 09:21:55.225663 140447344932736 d

In [0]:
MODEL_DIR = "bert_model"
tf.gfile.MkDir(MODEL_DIR)

In [0]:
# hyperparameters for BERT BASE

bert_base_config = {
    "attention_probs_dropout_prob": 0.1,
    "directionality": "bidi",
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "max_position_embeddings": 512,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "pooler_fc_size": 768,
    "pooler_num_attention_heads": 12,
    "pooler_num_fc_layers": 3,
    "pooler_size_per_head": 128,
    "pooler_type": "first_token_transform",
    "vocab_size": 30522
}



In [0]:

with open("{}/bert_config.json".format(MODEL_DIR), "w") as base:
    json.dump(bert_base_config, base, indent=2)
    
with open("{}/bert_vocab".format(MODEL_DIR), "w") as vocab:
    vocab_bert = open(VOCAB_FILE, 'r').read()
    vocab.write(vocab_bert)
    

In [0]:
!cp -r $MODEL_DIR $PRETRAINING_DIR /mydrive/

In [0]:
#TO DO
#code to transfer latest checkpoint from another directory to new directory



In [135]:
TRAIN_BATCH_SIZE = 128
MAX_PREDICTIONS =20
MAX_SEQ_LENGTH = 128
MASKED_LM_PROB = 0.15

EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 1000
SAVE_CHECKPOINTS_STEPS = 100
NUM_TPU_CORES = 8

BERT_DRIVE_DIR = "{}/{}".format('/mydrive', MODEL_DIR)
DATA_DRIVE_DIR = "{}/{}".format('/mydrive', PRETRAINING_DIR)

PATH_TO_CHECKPOINT = os.path.join(BERT_DRIVE_DIR, "bert_model.ckpt")

INIT_CHECKPOINT = tf.train.latest_checkpoint(PATH_TO_CHECKPOINT)

if INIT_CHECKPOINT == None:
    print("no checkpoint found, loading the default")
    INIT_CHECKPOINT = tf.train.load_checkpoint(PATH_TO_CHECKPOINT)

CONFIG_FILE = os.path.join(BERT_DRIVE_DIR, "bert_config.json")
VOCAB_FILE = os.path.join(BERT_DRIVE_DIR, "bert_vocab")

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_DRIVE_DIR, '*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

2019-06-26 11:29:31,331 :     Using checkpoint: <tensorflow.python.pywrap_tensorflow_internal.CheckpointReader; proxy of <Swig Object of type 'tensorflow::checkpoint::CheckpointReader *' at 0x7f7dba2c1a20> >
I0626 11:29:31.331611 140179299624832 interactiveshell.py:2882] Using checkpoint: <tensorflow.python.pywrap_tensorflow_internal.CheckpointReader; proxy of <Swig Object of type 'tensorflow::checkpoint::CheckpointReader *' at 0x7f7dba2c1a20> >
2019-06-26 11:29:31,335 :     Using 1 data shards
I0626 11:29:31.335389 140179299624832 interactiveshell.py:2882] Using 1 data shards


no checkpoint found, loading the default


In [0]:
model_fn = model_fn_builder(
    bert_config = bert_config,
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=TRAIN_STEPS,
    num_warmup_steps=10,
    use_tpu=USE_TPU,
    use_one_hot_embeddings=True)


tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
            cluster=tpu_cluster_resolver,
            model_dir=BERT_DRIVE_DIR,
            save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
            tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
            num_shards=NUM_TPU_CORES,
            per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

train_input_fn = input_fn_builder(
    input_files=input_files,
    max_seq_length=MAX_SEQ_LENGTH,
    max_predictions_per_seq=MAX_PREDICTIONS,
    is_training=True)


In [0]:
estimator.train(input_fn=train_input_fn, max_steps = TRAIN_STEPS )

In [141]:
!python3 bert/run_pretraining.py \
--input_file=$input_files \
--output_dir=$BERT_DRIVE_DIR \
--do_train=True \
--do_eval=True \
--bert_config_file=$CONFIG_FILE \
--init_checkpoint=$INIT_CHECKPOINT \
--train_batch_size=128 \
--max_seq_length=128 \
--num_train_steps=1000 \
--num_warmup_steps=10 \
--learning_rate=2e-5 \
--use_tpu=True

/bin/bash: -c: line 0: syntax error near unexpected token `>'
/bin/bash: -c: line 0: `python3 bert/run_pretraining.py --input_file=['/mydrive/pretraining_data/shard_0000.tfrecord'] --output_dir=/mydrive/bert_model --do_train=True --do_eval=True --bert_config_file=/mydrive/bert_model/bert_config.json --init_checkpoint=<tensorflow.python.pywrap_tensorflow_internal.CheckpointReader; proxy of <Swig Object of type 'tensorflow::checkpoint::CheckpointReader *' at 0x7f7dba2c1a20> > --train_batch_size=128 --max_seq_length=128 --num_train_steps=1000 --num_warmup_steps=10 --learning_rate=2e-5 --use_tpu=True'
